In [108]:
import numpy as np
import torch
import torchvision as tv
import os
import time
from __future__ import print_function

In [49]:
'''Some code pieces taken from example: https://github.com/pytorch/examples/blob/42e5b996718797e45c46a25c55b031e6768f8440/imagenet/main.py#L89-L101'''

'Some code pieces taken from example: https://github.com/pytorch/examples/blob/42e5b996718797e45c46a25c55b031e6768f8440/imagenet/main.py#L89-L101'

In [150]:
args = {'num_classes': 4,
        'batch_size': 16,
        'lr': 0.01,
        'workers': 1,
        'momentum': 0.9,
        'weight_decay': 0.1,
        'start_epoch': 0,
        'epochs': 10,
        'print_freq': 1}

In [153]:
class CustomDenseNet(torch.nn.Module):
    def __init__(self):
        super(CustomDenseNet, self).__init__()
        densenet = tv.models.densenet121(pretrained=True)
        densenet.classifier = torch.nn.Linear(1024, args['num_classes'])
        self.densenet = densenet
        self.softmax = torch.nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.densenet(x)
        x = self.softmax(x)
        return x
    
model = CustomDenseNet()
for name, module, in model.named_parameters():
    print(name)

densenet.features.conv0.weight
densenet.features.norm0.weight
densenet.features.norm0.bias
densenet.features.denseblock1.denselayer1.norm.1.weight
densenet.features.denseblock1.denselayer1.norm.1.bias
densenet.features.denseblock1.denselayer1.conv.1.weight
densenet.features.denseblock1.denselayer1.norm.2.weight
densenet.features.denseblock1.denselayer1.norm.2.bias
densenet.features.denseblock1.denselayer1.conv.2.weight
densenet.features.denseblock1.denselayer2.norm.1.weight
densenet.features.denseblock1.denselayer2.norm.1.bias
densenet.features.denseblock1.denselayer2.conv.1.weight
densenet.features.denseblock1.denselayer2.norm.2.weight
densenet.features.denseblock1.denselayer2.norm.2.bias
densenet.features.denseblock1.denselayer2.conv.2.weight
densenet.features.denseblock1.denselayer3.norm.1.weight
densenet.features.denseblock1.denselayer3.norm.1.bias
densenet.features.denseblock1.denselayer3.conv.1.weight
densenet.features.denseblock1.denselayer3.norm.2.weight
densenet.features.dense

In [154]:
def train(train_loader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    model.train()
    end = time.time()
    for i, (input, target) in enumerate(train_loader):
        data_time.update(time.time() - end)

        target = target#.cuda(async=True)
        input_var = torch.autograd.Variable(input)
        target_var = torch.autograd.Variable(target)

        output = model(input_var)
        loss = criterion(output, target_var)

        prec1, prec5 = accuracy(output.data, target, topk=(1, 3))
        prec1 = accuracy(output.data, target, topk=(1,))
        losses.update(loss.data[0], input.size(0))
        top1.update(prec1[0], input.size(0))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        batch_time.update(time.time() - end)
        end = time.time()
        if i % args['print_freq'] == 0:
            print("Epoch %d [%d/%d], Time: %.f, Loss: %.4f, Precision: %.2f" %\
                     (epoch, i, len(train_loader), batch_time.val, loss, prec1[0]))

In [156]:
def validate(val_loader, model, criterion):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    model.eval()
    end = time.time()
    for i, (input, target) in enumerate(val_loader):
        target = target.cuda(async=True)
        input_var = torch.autograd.Variable(input, volatile=True)
        target_var = torch.autograd.Variable(target, volatile=True)

        output = model(input_var)
        loss = criterion(output, target_var)

        prec1 = accuracy(output.data, target)
        losses.update(loss.data[0], input.size(0))
        top1.update(prec1[0], input.size(0))

        batch_time.update(time.time() - end)
        end = time.time()

        if i % args.print_freq == 0:
            print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))
    print(' * Prec@1 {top1.avg:.3f}'
          .format(top1=top1))

    return top1.avg

In [157]:
current_dir = os.getcwd()
data_path = current_dir + '/WeatherImageDataset'
train_dir = data_path + '/train'
val_dir = data_path + '/val'
normalize = tv.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
train_loader = torch.utils.data.DataLoader(
    tv.datasets.ImageFolder(train_dir, tv.transforms.Compose([
        tv.transforms.RandomSizedCrop(224),
        tv.transforms.RandomHorizontalFlip(),
        tv.transforms.ToTensor(),
        normalize,
    ])),
    batch_size=args['batch_size'], shuffle=True,
    num_workers=args['workers'])#, pin_memory=True)

val_loader = torch.utils.data.DataLoader(
    tv.datasets.ImageFolder(val_dir, tv.transforms.Compose([
        tv.transforms.Scale(256),
        tv.transforms.CenterCrop(224),
        tv.transforms.ToTensor(),
        normalize,
    ])),
    batch_size=args['batch_size'], shuffle=False,
    num_workers=args['workers'])#, pin_memory=True)

criterion = torch.nn.CrossEntropyLoss()#.cuda()
optimizer = torch.optim.SGD(model.parameters(), args['lr'],
                            momentum=args['momentum'],
                            weight_decay=args['weight_decay'])

# if args.evaluate:
#     validate(val_loader, model, criterion)

for epoch in range(args['start_epoch'], args['epochs']):
    adjust_learning_rate(optimizer, epoch)

    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch)

    # evaluate on validation set
    prec1 = validate(val_loader, model, criterion)

    # remember best prec@1 and save checkpoint
    is_best = prec1 > best_prec1
    best_prec1 = max(prec1, best_prec1)
#     save_checkpoint({
#         'epoch': epoch + 1,
#         #'arch': args.arch,
#         'state_dict': model.state_dict(),
#         'best_prec1': best_prec1,
#     }, is_best)

Epoch 0 [0/579], Time: 40, Loss: 1.2225, Precision: 37.50
Epoch 0 [1/579], Time: 48, Loss: 1.1889, Precision: 50.00


Process Process-38:
KeyboardInterrupt
Traceback (most recent call last):
  File "/Users/Anuar_The_Great/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/Anuar_The_Great/anaconda/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/Anuar_The_Great/anaconda/lib/python2.7/site-packages/torch/utils/data/dataloader.py", line 36, in _worker_loop
    r = index_queue.get()
  File "/Users/Anuar_The_Great/anaconda/lib/python2.7/multiprocessing/queues.py", line 378, in get
    return recv()
  File "/Users/Anuar_The_Great/anaconda/lib/python2.7/site-packages/torch/multiprocessing/queue.py", line 21, in recv
    buf = self.recv_bytes()


KeyboardInterrupt: 

In [69]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = args['lr'] * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res